In [39]:
businesses = sc.textFile('../YelpDataset/yelp_academic_dataset_business.json')
reviews = sc.textFile('../YelpDataset/yelp_academic_dataset_review.json')
users = sc.textFile('../YelpDataset/yelp_academic_dataset_user.json')
# reviews.first()
businesses.first()
# users.first()

u'{"business_id": "5UmKMjUEUNdYWqANhGckJw", "full_address": "4734 Lebanon Church Rd\\nDravosburg, PA 15034", "hours": {"Friday": {"close": "21:00", "open": "11:00"}, "Tuesday": {"close": "21:00", "open": "11:00"}, "Thursday": {"close": "21:00", "open": "11:00"}, "Wednesday": {"close": "21:00", "open": "11:00"}, "Monday": {"close": "21:00", "open": "11:00"}}, "open": true, "categories": ["Fast Food", "Restaurants"], "city": "Dravosburg", "review_count": 4, "name": "Mr Hoagie", "neighborhoods": [], "longitude": -79.9007057, "state": "PA", "stars": 4.5, "latitude": 40.3543266, "attributes": {"Take-out": true, "Drive-Thru": false, "Good For": {"dessert": false, "latenight": false, "lunch": false, "dinner": false, "brunch": false, "breakfast": false}, "Caters": false, "Noise Level": "average", "Takes Reservations": false, "Delivery": false, "Ambience": {"romantic": false, "intimate": false, "classy": false, "hipster": false, "divey": false, "touristy": false, "trendy": false, "upscale": fal

In [157]:
import json, operator

selectedCategories = ['Restaurants', 'Shopping', 'Beauty & Spas', 'Bars', 'Automotive', 'Event Planning & Services',
                     'Fast Food', 'Coffee & Tea', 'Hotels', 'Real Estate', 'Dentists', 'Gyms']

def mapper_reviews_filter(rows):
    for row in rows:
        row = json.loads(row)
        yield (row['user_id'], (row['business_id'], row['stars']))

def mapper_buisness_filter(rows):
    for row in rows:
        row = json.loads(row)
        for category in row['categories']:
            if(category in selectedCategories):
                yield (row['business_id'], category)

def mapper_users_filter(rows):
    for row in rows:
        row = json.loads(row)
        elite = 0
        if len(row['elite']) == 0:
            elite = 1
        else:
            elite = 4 + len(row['elite'])
        yield (row['user_id'], elite)

def mapper_buisness_rating(rows):
    for row in rows:
        buisness_id = row[0][0]
        review_rating = row[0][1]
        elite_status = row[1]
        yield (buisness_id, review_rating * elite_status)        

def mapper_normalized_total(records):
    for category in records:
        maxCheckins = max(category[1], key=operator.itemgetter(1))[1]
        minCheckins = min(category[1], key=operator.itemgetter(1))[1]
        for buisness in category[1]:
            normalizedTotal = (float(buisness[1]) - minCheckins)/(maxCheckins - minCheckins) * 50
            yield (category[0], (buisness[0], normalizedTotal))        
        
reviews_filtered = reviews.mapPartitions(mapper_reviews_filter)
buisness_filtered = businesses.mapPartitions(mapper_buisness_filter)
users_filtered = users.mapPartitions(mapper_users_filter)
joined = reviews_filtered.join(users_filtered).map(lambda x: x[1])
total = joined.mapPartitions(mapper_buisness_rating).groupByKey().map(lambda x: (x[0], sum(x[1])))
category_join = total.join(buisness_filtered).map(lambda x: (x[1][1], (x[0], x[1][0]))).groupByKey().mapValues(list)
total_normalized = category_join.mapPartitions(mapper_normalized_total) \
                                .groupByKey().mapValues(list) \
                                .map(lambda (k, v): (k, sorted(v, key=lambda x: x[1], reverse=True)))
total_normalized.take(3)



[(u'Restaurants',
  [(u'2e2e7WgqU1BnpxmQL5jbfw', 50.0),
   (u'4bEjOyTaDG24SY5TxsaUNQ', 45.55776830925034),
   (u'zt1TpTuJ6y9n551sw9TaEg', 40.6068605656416),
   (u'sIyHTizqAiGu12XMLX3N3g', 35.225237742373714),
   (u'YNQgak-ZLtYJQxlDwN-qIg', 32.164690626157835),
   (u'tFU2Js_nbIZOrnKfYJYBBg', 31.66604915400766),
   (u'eq6lQI039SBLC6sHm3idGA', 28.840156848215386),
   (u'CZjcFdvJhksq9dy58NVEzw', 27.336513523527234),
   (u'Xhg93cMdemu5pAMkDoEdtQ', 24.68506854390515),
   (u'aGbjLWzcrnEx2ZmMCFm3EA', 23.991910584166977),
   (u'lliksv-tglfUz1T3B3vgvA', 23.295665061133754),
   (u'DO3Gk17RyJVW7zYMCtYPnw', 22.793164134864767),
   (u'QbmcCE_cLq4WO8ZMKImaLw', 22.428059775225393),
   (u'xyTJYlbE_MLouK6rCou6zg', 20.629554155860195),
   (u'BqrTtox0JbG-P_DKBB5bBw', 19.998919352846734),
   (u'jOuERtVf7QePnK9ZcdH5XA', 19.3211992095838),
   (u'PXviRcHR1mqdH4vRc2LEAQ', 18.894343584043476),
   (u'xfwRO04KbAPw_zRotCfWQQ', 18.735334074348525),
   (u'vxxMqBaAHuWdx4impsLSSA', 16.423521057181674),
   (u'HbUQ_3dlm